# Taguchi Array

In [76]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from utils import validate_parameters, score_vals, efficiency_calc

In [77]:
MODULE = 0.8e-3
MC = 0.8467e-3

## Input parameter values
#### Currently assumes we will use values from the paper (added to the dict seperately)

In [78]:
# Previous (deprecated) values
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    # 'z_sh' : [9, 8, 7][::-1],
    # 'z_r2' : [83, 82, 80][::-1],
    # 'x_s' : [-0.5, 0, 1],
    # 'x_r2' : [-0.5, 0, 0.5],
    'z_sh' : [10, 11, 15],
    'z_r2' : [70, 71, 72],
    'x_s' : [0.9 , 0.4, 0.3],
    'x_r2' : [-1.  , -0.45, -0.55],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 8, 6][::-1],
}

# Include reference paper values as 1 set
paper_vals = [6, 81, .5, 1, 0.1e-3, 10] # NOTE - 6 = 12 (z_s) / 2.

# Put the dictionary into a vector & count total possible designs
quick_param = []
for i in (range(2,len(param_vals))):
    key_i = list(param_vals.keys())[i]
    param_vals[key_i].insert(0, paper_vals[i-2])
    quick_param.append(param_vals[key_i])

# ALL parameters will create gears that work:
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [9, 10, 11, 15],
    'z_r2' : [70, 71, 72, 73],
    'x_s' : [1, 0.9 , 0.8, 0.4],
    'x_r2' : [-0.25, -0.35  , -0.45, -0.55],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3, 0.1e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 10, 8, 6][::-1],
}
# Better gear ratios while all still working (BayZop):
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [6, 7, 8, 9],
    'z_r2' : [72, 73, 74, 75],
    'x_s' : [1.0, 0.9 , 0.8, -1],
    'x_r2' : [1, 0.9, 0.8, 0.7],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3, 0.1e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 10, 8, 6][::-1],
}

# 'Brute Force' values for maximizing gear-ratio score:
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [6, 7, 8, 9],
    'z_r2' : [76, 78, 81, 83],
    'x_s' : [-0.5, -0.25, 0.5, 0.75],
    'x_r2' : [-0.45, 0, 0.85, 1.0],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3, 0.1e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'g_thickness' : [12, 10, 8, 6][::-1], # gear thickness
}

In [79]:
# Designate target parameter values
# in_vals = [z_s, z_r2, x_s, x_r2, Cl, gear_thickness] # Cl_i = clearance (i = 1,2,3)


# Inclusion of offset values, AGRESSIVE SEARCH (Expect success on 11-18 of 32, success up to ~145(80% efficiency)):
offsets = [-1, 0, 1, 2] # p1_offset, p2_offset
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [6, 7, 8, 9],
    'z_r2' : [76, 81, 83, 85],
    'x_s' : [-0.8, -0.5, 0, 0.5],
    'x_r2' : [-0.25, 0, 0.5, 1],
    'Cl' : [0.3e-3, 0.2e-3, 0.1e-3, 0][::-1], #NOTE - 0.4 is theoretical max, but is likely too large.
    'g_thickness' : [12, 10, 8, 6][::-1], # gear thickness
    'p1_offset' : offsets,
    'p2_offset' : offsets
}

# # Broad/Naiive search (Expect success on 11-22 of 32, success up to ~207 (.14 efficiency)):
# offsets = [-1, 0, 1, 2] # p1_offset, p2_offset
# param_vals = {
#     'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
#     'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
#     'z_sh' : [6, 8, 10, 12],
#     'z_r2' : [73, 77, 81, 85],
#     'x_s' : [-1, -0.5 , 0.5, 1],
#     'x_r2' : [-1, -0.5 , 0.5, 1],
#     'Cl' : [0.3e-3, 0.2e-3, 0.1e-3, 0][::-1], #NOTE - 0.4 is theoretical max, but is likely too large.
#     'g_thickness' : [12, 10, 8, 6][::-1], # gear thickness
#     'p1_offset' : offsets,
#     'p2_offset' : offsets
# }

# # BayZop informed search (Expect success on 17-24 of 32, success up to ~271 (.35 efficiency)):
# offsets = [-1, 0, 1, 2] # p1_offset, p2_offset
# param_vals = {
#     'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
#     'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
#     'z_sh' : [9, 8, 7, 6],
#     'z_r2' : [81, 80, 78, 77],
#     'x_s' : [ 0.066, -0.034, -0.134, -0.8],
#     'x_r2' : [0.262,  0.162,  0.062, -0.038],
#     'Cl' : [0.3e-3, 0.2e-3, 0.1e-3, 0][::-1], #NOTE - 0.4 is theoretical max, but is likely too large.
#     'g_thickness' : [12, 10, 8, 6][::-1], # gear thickness
#     'p1_offset' : offsets,
#     'p2_offset' : offsets
# }

headers = list(param_vals.keys())
rows = list(zip(*param_vals.values()))

print(tabulate(rows, headers=headers, tablefmt="grid"))

design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param[:-1])

design_vals = np.array(design_vals)

print(f'Design vals shape: {design_vals.shape}') # Note that 8 parameters yield 9 geometric design values

+------------+-------------+--------+--------+-------+--------+--------+---------------+-------------+-------------+
| material   | lubricant   |   z_sh |   z_r2 |   x_s |   x_r2 |     Cl |   g_thickness |   p1_offset |   p2_offset |
+============+=============+========+========+=======+========+========+===============+=============+=============+
| PLA        | None        |      6 |     76 |  -0.8 |  -0.25 | 0      |             6 |          -1 |          -1 |
+------------+-------------+--------+--------+-------+--------+--------+---------------+-------------+-------------+
| ABS        | LI          |      7 |     81 |  -0.5 |   0    | 0.0001 |             8 |           0 |           0 |
+------------+-------------+--------+--------+-------+--------+--------+---------------+-------------+-------------+
| --         | Oil         |      8 |     83 |   0   |   0.5  | 0.0002 |            10 |           1 |           1 |
+------------+-------------+--------+--------+-------+--------+-

## Use L32b Taguchi array to prepare design parameters for each test

In [80]:
Taguchi = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
[1, 1, 3, 3, 3, 3, 3, 3, 3, 3],
[1, 1, 4, 4, 4, 4, 4, 4, 4, 4],
[1, 2, 1, 1, 2, 2, 3, 3, 4, 4],
[1, 2, 2, 2, 1, 1, 4, 4, 3, 3],
[1, 2, 3, 3, 4, 4, 1, 1, 2, 2],
[1, 2, 4, 4, 3, 3, 2, 2, 1, 1],
[1, 3, 1, 2, 3, 4, 1, 2, 3, 4],
[1, 3, 2, 1, 4, 3, 2, 1, 4, 3],
[1, 3, 3, 4, 1, 2, 3, 4, 1, 2],
[1, 3, 4, 3, 2, 1, 4, 3, 2, 1],
[1, 4, 1, 2, 4, 3, 3, 4, 2, 1],
[1, 4, 2, 1, 3, 4, 4, 3, 1, 2],
[1, 4, 3, 4, 2, 1, 1, 2, 4, 3],
[1, 4, 4, 3, 1, 2, 2, 1, 3, 4],
[2, 1, 1, 4, 1, 4, 2, 3, 2, 3],
[2, 1, 2, 3, 2, 3, 1, 4, 1, 4],
[2, 1, 3, 2, 3, 2, 4, 1, 4, 1],
[2, 1, 4, 1, 4, 1, 3, 2, 3, 2],
[2, 2, 1, 4, 2, 3, 4, 1, 3, 2],
[2, 2, 2, 3, 1, 4, 3, 2, 4, 1],
[2, 2, 3, 2, 4, 1, 2, 3, 1, 4],
[2, 2, 4, 1, 3, 2, 1, 4, 2, 3],
[2, 3, 1, 3, 3, 1, 2, 4, 4, 2],
[2, 3, 2, 4, 4, 2, 1, 3, 3, 1],
[2, 3, 3, 1, 1, 3, 4, 2, 2, 4],
[2, 3, 4, 2, 2, 4, 3, 1, 1, 3],
[2, 4, 1, 3, 4, 2, 4, 2, 1, 3],
[2, 4, 2, 4, 3, 1, 3, 1, 2, 4],
[2, 4, 3, 1, 2, 4, 2, 4, 3, 1],
[2, 4, 4, 2, 1, 3, 1, 3, 4, 2],
])-1

In [81]:
# Designate a MU for the efficiency estimation
MU = 0.08

In [82]:


added_cols = ['eta_f', 'eta_b', 'gear ratio']
param_cols = ['Design'] + list(param_vals.keys()) + added_cols
# need to make a list (columns) of tuples (rows)
test_params = []
# use the Taguchi array to get the parameter values
for i in range(len(Taguchi)):
    Tag_vals = Taguchi[i]
    new_params = tuple()
    for ind, val in enumerate(Tag_vals):
        key_i = list(param_vals.keys())[ind]
        new_params = new_params + (param_vals[key_i][val],)
    
    # Solve estimated efficiency (validate takes )
    design_vals, _, _, _, ratio = validate_parameters(new_params[2:-3], offsets=new_params[-2:])
    # design_vals, _, _, _, ratio = validate_parameters(new_params[2:-3])
    eta_f, eta_b = efficiency_calc(design_vals, mu = MU)

    new_params = new_params + (eta_f, eta_b, ratio)
    new_params = (i+1,) + new_params
    test_params.append(new_params)   

print(tabulate(test_params, headers=param_cols, tablefmt="grid"))

# how many eta_b are 'viable' (above 0.3)?
viable_count = 0
for i in range(len(test_params)):
    if test_params[i][-2] > 0.3:
        viable_count += 1
print(f'Presumably viable designs: {viable_count} out of {len(test_params)}')

+----------+------------+-------------+--------+--------+-------+--------+--------+---------------+-------------+-------------+-----------+----------+--------------+
|   Design | material   | lubricant   |   z_sh |   z_r2 |   x_s |   x_r2 |     Cl |   g_thickness |   p1_offset |   p2_offset |     eta_f |    eta_b |   gear ratio |
+==========+============+=============+========+========+=======+========+========+===============+=============+=============+===========+==========+==============+
|        1 | PLA        | None        |      6 |     76 |  -0.8 |  -0.25 | 0      |             6 |          -1 |          -1 |  0.680316 | 0.573201 |      53.5983 |
+----------+------------+-------------+--------+--------+-------+--------+--------+---------------+-------------+-------------+-----------+----------+--------------+
|        2 | PLA        | None        |      7 |     81 |  -0.5 |   0    | 0.0001 |             8 |           0 |           0 |  0.743238 | 0.669858 |     115.481  |
+---

## Calculate design (CAD) parameters from design parameters

In [83]:
test_params[8][5:11]

(0, 1, 0, 8, 1, 2)

In [84]:
# design features:
d_cols = ['Design', 'z_s', 'z_p1', 'z_p2', 'z_r2', 'xs', 'xp1', 'xp2', 'xr2', 'r_a [m]', 'gear_thickness']

# need to make a list (columns) of tuples (rows)
d_params = []
for j in range(len(test_params)):
    quick_param = list(test_params[j])[3:8]
    design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param, offsets=test_params[j][9:11])
    design_vals = np.array(design_vals)
    design_vals[0] = design_vals[0] * 2. # z_s is 2*z_sh
    design_vals = np.vstack([design_vals,[test_params[j][8]]]) # gear thickness
    # add j+1 to the front of design_vals
    design_vals = np.insert(design_vals, 0, [j+1])
    # round for visual
    design_vals = np.round(design_vals, 4)
    new_d = tuple(design_vals)
    # for i in range(len(d_cols)):
    #     new_d = new_d + (design_vals[i])
    d_params.append(new_d)

print(tabulate(d_params, headers=d_cols, tablefmt="grid"))

+----------+-------+--------+--------+--------+------+--------+---------+-------+-----------+------------------+
|   Design |   z_s |   z_p1 |   z_p2 |   z_r2 |   xs |    xp1 |     xp2 |   xr2 |   r_a [m] |   gear_thickness |
+==========+=======+========+========+========+======+========+=========+=======+===========+==================+
|        1 |    12 |     38 |     27 |     76 | -0.8 |  0.9   |  0.5344 | -0.25 |    0.0201 |                6 |
+----------+-------+--------+--------+--------+------+--------+---------+-------+-----------+------------------+
|        2 |    14 |     38 |     32 |     81 | -0.5 |  0.125 |  0.0521 |  0    |    0.0206 |                8 |
+----------+-------+--------+--------+--------+------+--------+---------+-------+-----------+------------------+
|        3 |    16 |     37 |     34 |     83 |  0   | -0.25  | -0.2746 |  0.5  |    0.0212 |               10 |
+----------+-------+--------+--------+--------+------+--------+---------+-------+-----------+---